#  $T(n) = \sqrt{n} \cdot T(\sqrt{n}) + n$

Let

$$
U(n)\;=\;\frac{T(n)}{n^n}.
$$

Then from

$$
T(n)=2^n\,T\!\left(\frac n2\right)+n^n
$$

we get

$$
U(n)\;=\;\frac{2^n\,T(n/2)}{n^n}+1
\;=\;2^n\cdot \frac{U(n/2)\,(n/2)^{\,n/2}}{n^n}+1
\;=\;\Big(\frac{2}{n}\Big)^{\!n/2}\,U\!\left(\frac n2\right)+1.
$$

So the normalized recurrence is

$$
\boxed{\,U(n)=\Big(\tfrac{2}{n}\Big)^{n/2}U(n/2)+1\,}.
$$

Key observation: for $n\ge 4$,

$$
\Big(\tfrac{2}{n}\Big)^{n/2}\le \Big(\tfrac{1}{2}\Big)^2=\tfrac14,
$$

and it decays **super-exponentially** with $n$. Unrolling $k$ levels (assume $n$ is a power of 2 for clarity),

$$
U(n)=1+\Big(\tfrac{2}{n}\Big)^{n/2}
+\Big(\tfrac{2}{n/2}\Big)^{n/4}
+\Big(\tfrac{2}{n/4}\Big)^{n/8}
+\cdots
+\Big[\prod_{i=0}^{k-1}\Big(\tfrac{2}{n/2^i}\Big)^{n/2^{i+1}}\Big]\,
U\!\left(\tfrac{n}{2^k}\right).
$$

Each term is tiny:

$$
\Big(\tfrac{2}{n/2^i}\Big)^{n/2^{i+1}}
=\Big(\tfrac{2^{i+1}}{n}\Big)^{n/2^{i+1}}
=\exp\!\Big(\tfrac{n}{2^{i+1}}\big((i+1)\ln2-\ln n\big)\Big),
$$

whose exponent is large and negative for all $i\le \log_2 n -1$. Thus the tail is dominated by the **first** correction
$\big(\tfrac{2}{n}\big)^{n/2}$, giving

$$
U(n)=1+O\!\left(\Big(\tfrac{2}{n}\Big)^{n/2}\right).
$$

Multiplying back,

$$
\boxed{\,T(n)=n^n\Big(1+O\!\big((2/n)^{n/2}\big)\Big)=\Theta(n^n).\,}
$$

$T(n)$ is $n^n$ up to a vanishingly small relative error.


---

## What about the splits?

Given 

  $$
  T(n)=2^n\,T(n/2)+n^n,
  $$

  the **“$2^n$” is a multiplier**, not a branching factor. There is **one** recursive call per level (to size $n/2$). So the “tree” is actually a **single chain** of depth $\log_2 n$. Each edge just carries a weight:

  $$
  n \xrightarrow{\times\,2^{n}} \frac{n}{2} \xrightarrow{\times\,2^{n/2}} \frac{n}{4} \xrightarrow{\times\,2^{n/4}} \cdots
  $$

Contrast with a typical Master-theorem form like $T(n)=a\,T(n/b)+f(n)$. There the number of subproblems per node is $a$ (e.g., $a=2$ gives a binary tree with $2^j$ nodes at level $j$). Here, $a=1$; only the cost of the **single** child is scaled by a big factor.

In [ ]:
# Fix mathtext error by using plain text for "#nodes" annotations.
import matplotlib.pyplot as plt
import numpy as np

def draw_weighted_chain_with_edges(levels=6, fname="weighted_chain_tree.svg"):
    fig = plt.figure(figsize=(8, 10))
    ax = fig.add_subplot(111)
    ax.set_axis_off()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1.05)

    xs = 0.5*np.ones(levels)
    ys = np.linspace(0.95, 0.05, levels)

    for j in range(levels-1):
        ax.plot([xs[j], xs[j+1]], [ys[j], ys[j+1]])
        if j == 0:
            lbl = "× 2^n"
        elif j == 1:
            lbl = "× 2^{n/2}"
        elif j == 2:
            lbl = "× 2^{n/4}"
        else:
            lbl = f"× 2^(n/2^{j})"
        mx = (xs[j]+xs[j+1])/2 + 0.03
        my = (ys[j]+ys[j+1])/2
        ax.text(mx, my, lbl, ha="left", va="center", fontsize=11)

    for j in range(levels):
        ax.scatter([xs[j]], [ys[j]], s=180, zorder=3)
        if j == 0:
            node = "n"
            cost = "n^n"
        else:
            node = f"n/2^{j}"
            cost = f"(n/2^{j})^(n/2^{j})"

        ax.text(xs[j]-0.12, ys[j], node, ha="right", va="center", fontsize=14)
        ax.text(xs[j]+0.03, ys[j]+0.03, cost, ha="left", va="bottom", fontsize=11)

    ax.text(0.5, 1.01,
            "Weighted single-branch: T(n)=2^n T(n/2)+n^n (edge labels are multipliers)",
            ha="center", va="bottom", fontsize=14)
    ax.text(0.5, 0.02,
            "No branching: one subproblem per level; depth ≈ log2 n; multipliers accumulate.",
            ha="center", va="bottom", fontsize=11)

    fig.tight_layout()
    fig.savefig(fname, format="svg")
    plt.close(fig)


draw_branching_example()
draw_weighted_chain_with_edges()

from IPython.display import HTML
HTML("""
<div style="display:flex; gap:12px;">
  <img src="weighted_chain_tree.svg" style="width:48%;">
</div>
""")

display(HTML("weighted_chain_tree.svg"))


---

# Variable Changing in Recurrence Relations

## Why Change Variables?

Some recurrences look messy but simplify dramatically once we look at them through the right “lens.” Changing variables means analyzing a transformed version of $T(n)$—like $T(n)/n$, $T(n)/n^2$, or $\log T(n)$—that reveals hidden structure.

Key idea: **don’t fight the recurrence head-on; tilt your perspective so the pattern becomes obvious.**

---

## Example 1: Merge Sort — the Classic Transformation

We start with the merge sort recurrence:

$$
T(n) = 2T(n/2) + n.
$$

Divide both sides by $n$:

$$
\frac{T(n)}{n} = 2 \cdot \frac{T(n/2)}{n} + 1.
$$

Notice $\tfrac{T(n/2)}{n} = \tfrac{T(n/2)}{n/2} \cdot \tfrac{1}{2}$. So:

$$
\frac{T(n)}{n} = \frac{2}{2} \cdot \frac{T(n/2)}{n/2} + 1 = S(n/2) + 1,
$$

if we define

$$
S(n) = \frac{T(n)}{n}.
$$

This gives a much simpler recurrence:

$$
S(n) = S(n/2) + 1.
$$

---

### Solving the Transformed Recurrence

Unroll it:

$$
S(n) = S(n/2) + 1 = S(n/4) + 2 = \cdots = S(1) + \log_2 n.
$$

So:

$$
S(n) = \Theta(\log n).
$$

Finally, undo the transformation:

$$
T(n) = n \cdot S(n) = \Theta(n \log n).
$$

**Lesson:** dividing by $n$ exposed the logarithmic growth hidden in the recurrence.

---

## General Strategy

1. **Look for structure**: Is the toll term $n$, $n^2$, $\log n$, or exponential?
2. **Pick a transformation** $S(n) = T(n)/f(n)$ where $f(n)$ matches the toll’s growth.
3. **Substitute and simplify**: Check if $S(n)$ has a simpler recurrence.
4. **Solve $S(n)$**, then reconstruct $T(n) = f(n),S(n)$.

---

## More Examples

### Example 2: Quadratic Toll

$$
T(n) = 4T(n/2) + n^2.
$$

Try $S(n) = T(n)/n^2$:

$$
S(n) = \frac{T(n/2)}{(n/2)^2} + 1 = S(n/2) + 1.
$$

Unroll:

$$
S(n) = S(1) + \log n.
$$

So:

$$
T(n) = n^2 \cdot (S(1) + \log n) = \Theta(n^2 \log n).
$$

---

### Example 3: Logarithmic Toll

$$
T(n) = 3T(n/3) + n \log n.
$$

Divide by $n$: let $S(n) = T(n)/n$:

$$
S(n) = S(n/3) + \log n.
$$

Unroll:

$$
S(n) = S(1) + \log n + \log (n/3) + \log (n/9) + \cdots.
$$

There are $\log\_3 n$ terms, each about $\log n$, so:

$$
S(n) = \Theta((\log n)^2),
$$

and

$$
T(n) = n \cdot S(n) = \Theta(n (\log n)^2).
$$

---

## Advanced Transformations

* **Divide by $\log n$**: for tolls like $c \log n$
* **Divide by $a^n$**: for exponential recurrences $T(n) = aT(n-1) + f(n)$
* **Divide by $n!$**: for factorial recurrences $T(n) = nT(n-1) + g(n)$
* **Take logarithms**: for multiplicative recurrences, e.g. $T(n) = T(n/2)^2 + n$

---

## Summary Table

| Recurrence          | Transformation | Simpler Form                      | Complexity                            |
| ------------------- | -------------- | --------------------------------- | ------------------------------------- |
| $aT(n/b)+cn$      | $T(n)/n$     | $S(n) = \tfrac{a}{b}S(n/b)+c$   | $\Theta(n\log n)$ (if $a=b$)      |
| $aT(n/b)+cn^k$    | $T(n)/n^k$   | $S(n) = \tfrac{a}{b^k}S(n/b)+c$ | $\Theta(n^k \log n)$ (if $a=b^k$) |
| $aT(n-1)+f(n)$    | $T(n)/a^n$   | $S(n)=S(n-1)+f(n)/a^n$          | exponential cases                     |
| $T(n)=T(n/2)^2+n$ | $\log T(n)$  | $S(n)=2S(n/2)+\log n$           | double-exponential                    |

---

## Pitfalls

1. **Base cases matter**: check that transformations preserve $T(1)$.
2. **Don’t mix totals with per-node values**: the recurrence gives one node’s cost, not the sum of the tree.
3. **Wrong guess of $f(n)$**: if the recurrence doesn’t simplify, try a different transformation.
4. **Algebra slips**: small mistakes compound — check each step carefully.

---

## Meta-Lesson

Changing variables is about finding the *right perspective*.